In [1]:
#导包
from netCDF4 import Dataset
from tensorflow.keras import layers, models
from tensorflow.keras.layers import ConvLSTM2D,LSTM, BatchNormalization, LayerNormalization,Input, Conv3D, TimeDistributed, Flatten
from tensorflow.keras.layers import Concatenate,Conv2D,TimeDistributed, MaxPooling2D, Input, MaxPooling3D
from tensorflow.keras.layers import  Reshape,multiply
from tensorflow.keras.layers import Layer,Lambda,Dot,ReLU, Dense, Dropout, Activation, Flatten,Attention
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import r2_score,accuracy_score,precision_score
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import regularizers
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, regularizers
# calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd 
import tensorflow as tf
import gc 
import gzip
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import match
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os 
%matplotlib inline

In [2]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

# 读取数据

## SSH

In [3]:
data_ssh = nc.Dataset(r'D:/data/SSH/SSH_2005-2015_month.nc')

In [4]:
time_ssh = data_ssh['time'][:]
print(len(time_ssh))
ssh_lat = data_ssh['lat'][:].data
ssh_lon = data_ssh['lon'][:].data
ssh = data_ssh['ssh'][:].data
print(ssh.shape)

132


In [ ]:
np.nanmax(ssh),np.nanmin(ssh)

## SSS

In [8]:
data_sss = nc.Dataset(r'D:/data/SSS/SSS_2005-2020_month.nc')

In [9]:
time_sss = data_sss['time'][:132]
print(len(time_sss))
sss_lat = data_sss['lat'][:].data
sss_lon = data_sss['lon'][:].data
sss = data_sss['sos'][:132].data 
print(sss.shape)
sss = np.squeeze(sss)
print(sss.shape)

132


In [12]:
np.nanmax(sss),np.nanmin(sss)

(36.072178, 13.604756)

## SSW

In [13]:
data_ssw = nc.Dataset(r'D:/data/SSW/SSW_2005-2018_month.nc')

In [14]:
time_ssw = data_ssw['time'][:132]
print(len(time_ssw))
lat_ssw = data_ssw['lat'][:].data
lon_ssw = data_ssw['lon'][:].data

132


In [16]:
uwnd = data_ssw['uwnd'][:132,314:594,440:1040].data  
vwnd = data_ssw['vwnd'][:132,314:594,440:1040].data
print(uwnd.shape)
print(vwnd.shape)

In [18]:
np.nanmax(uwnd),np.nanmin(uwnd),np.nanmax(vwnd),np.nanmin(vwnd)

(12.433594, -11.7988205, 8.8359995, -13.645481)

## SST 海表面温度

In [22]:
data_sst = nc.Dataset('D:/data/SST/SST_2005-2015_month.nc','r')

In [24]:
time_sst = data_sst['time'][:]
print(len(time_sst))
sst_lat = data_sst['lat'][:].data
sst_lon = data_sst['lon'][:].data
sst = data_sst['sst'][:].data
print(ssh.shape)

132


In [26]:
np.nanmax(sst),np.nanmin(sst)

(31.901611, -1.8474731)

## 3DS

In [27]:
data_3ds = nc.Dataset(r'D:/data/3DTS/3DS_2005-2015_month.nc')
sali_depth = data_3ds['so'][:].data
print(sali_depth.shape)

(132, 43, 280, 600)


In [28]:
depths = [0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
         450,500,550,600,700,800,900,1000,1100,1200,1300,1400,1500,1750,2000]
depths_use = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]

In [ ]:
sali_depth_use = []
selected_layers = []
for i, depth in enumerate(depths):
    if depth in depths_use:
        selected_layers.append(sali_depth[:, i:i+1, :, :])
if selected_layers:
    sali_depth_use = np.concatenate(selected_layers, axis=1)
else:
    sali_depth_use = None

In [31]:
print(sali_depth_use.shape)
sali_depth_use = np.transpose(sali_depth_use, (0,2,3,1))
print(sali_depth_use.shape)

(132, 33, 280, 600)
(132, 280, 600, 33)


## 数据合并

In [32]:
print(ssh.shape,sst.shape,uwnd.shape,vwnd.shape,sss.shape)
cat_data =np.stack((ssh,sst,uwnd,vwnd,sss),axis=3)
cat_data.shape

(132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600) (132, 280, 600)


(132, 280, 600, 5)

In [33]:
cat_data_ss_depth = np.concatenate((cat_data,sali_depth_use),axis=-1)
cat_data_ss_depth.shape 

## 将填充值赋值为NAN，且一个点中有一个维度为nan，则将这个点的所有维度都设置为nan 


In [35]:
cat_data_ss_depth_reshape  = np.transpose(cat_data_ss_depth,(1,2,0,3))
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape ((-1,132*38))
print(cat_data_ss_depth_reshape.shape)

(280, 600, 132, 38)
(168000, 5016)


In [36]:
cat_data_ss_depth_reshape[cat_data_ss_depth_reshape == 32767] = np.nan
nan_mask = np.isnan(cat_data_ss_depth_reshape)
np.sum(nan_mask.any(axis=1) == True)

In [38]:
cat_data_ss_depth_reshape[nan_mask.any(axis=1) == True] = np.nan   
print('cat_data_ss_depth_reshape:',cat_data_ss_depth_reshape.shape)

cat_data_ss_depth_reshape: (168000, 5016)


In [39]:
np.sum(np.isnan(cat_data_ss_depth_reshape[:,4]))  

63543

In [40]:
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,132,38)
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = np.transpose(cat_data_ss_depth_reshape,(1,0,2))
print(cat_data_ss_depth_reshape.shape)
cat_data_ss_depth_reshape = cat_data_ss_depth_reshape.reshape(-1,280,600,38)
print(cat_data_ss_depth_reshape.shape)

(168000, 132, 38)
(132, 168000, 38)
(132, 280, 600, 38)


In [41]:
np.sum(np.isnan(cat_data_ss_depth_reshape[1:2,:,:,2:3]))

63543

## 将数据分成小块

In [42]:
cat_data_ss = cat_data_ss_depth_reshape[:,:,:,:5]
cat_data_depth = cat_data_ss_depth_reshape[:,:,:,5:]
print(cat_data_ss.shape,cat_data_depth.shape)

(132, 280, 600, 5) (132, 280, 600, 33)


In [43]:
np.sum(np.isnan(cat_data_depth[1:2,:,:,5:6]))

63543

In [44]:
import numpy as np
import pandas as pd

time_size, lat_size, lon_size, channel = cat_data_ss.shape   # （132，280，600，5）
          
box_size = 9 
step_size = 1  

out_lat_size = (lat_size - box_size) // step_size  + 1 
out_lon_size = (lon_size - box_size) // step_size  + 1 

print('time_size:',time_size,'lat_size:',lat_size,'lon_size:',lon_size) 
print(out_lat_size,out_lon_size)  #(272 592)

out_data = np.zeros((time_size, out_lat_size , out_lon_size, box_size, box_size, channel)) 
out_data_y = np.zeros((time_size, out_lat_size , out_lon_size, 33)) 

for t in range(time_size):
    sample_idx = 0
    for lat in range(0, lat_size - box_size +1, step_size):
        for lon in range(0, lon_size - box_size +1, step_size):
            lat_min = lat
            lat_max = lat + box_size
            lon_min = lon
            lon_max = lon + box_size
            box_data = cat_data_ss[t, lat_min:lat_max, lon_min:lon_max,:]
            out_data[t,lat,lon, :, :, :] = box_data
            box_data = []
            sample_idx += 1

print('out_data的形状',out_data.shape)
    
#==============================================================================#   

for t in range(time_size):
    sample_idx = 0
    for lat in range(4, lat_size-4, step_size):
        for lon in range(4, lon_size-4, step_size):
            box_data_y = cat_data_depth[t, lat:lat+1, lon:lon+1,:]
            out_data_y[t,lat-4,lon-4, :] = box_data_y
            box_data_y = []
            sample_idx += 1

print('out_data_y的形状',out_data_y.shape)

time_size: 132 lat_size: 280 lon_size: 600
272 592
out_data的形状 (132, 272, 592, 9, 9, 5)
out_data_y的形状 (132, 272, 592, 33)


## 删除缺失值多于一半的数据

### 获取NAN_mask 

In [46]:
out_data_reshape = out_data.reshape((132,-1,9,9,5))
print(out_data_reshape.shape) 
nan_mask_out_data = np.isnan(out_data_reshape)
print(nan_mask_out_data.shape)

(132, 161024, 9, 9, 5)
(132, 161024, 9, 9, 5)


In [47]:
np.count_nonzero(nan_mask_out_data[7:8,:,:,:,2:3])

4784733

In [48]:
nan_mask_out_data_one = nan_mask_out_data[0,:,:,:,0]
nan_mask_out_data_one = nan_mask_out_data_one.reshape(-1,9*9) 
np.count_nonzero(nan_mask_out_data_one)
nan_mask_out_data_one.shape
nan_mask_out_data_one

### 为有效值大于一半的方块标注True

In [50]:
nan_mask = [] 
for i in nan_mask_out_data_one:
    if(np.count_nonzero(i) <=40 and i[40] == False): 
        nan_mask.append(True)
    else:
        nan_mask.append(False)

In [51]:
np.count_nonzero(nan_mask)

101529

In [52]:
out_data_reshape =  np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y,(1,2,0,3))
print('out_data_y_reshape:',out_data_y_reshape.shape)
out_data_y_reshape = out_data_y_reshape.reshape(-1,132,33)
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (161024, 132, 9, 9, 5)
out_data_y_reshape: (272, 592, 132, 33)
out_data_y_reshape: (161024, 132, 33)


## 开始删除无效数据

In [53]:
out_data_reshape = out_data_reshape[nan_mask]
out_data_y_reshape = out_data_y_reshape[nan_mask]
print(out_data_reshape.shape,out_data_y_reshape.shape)

(101529, 132, 9, 9, 5) (101529, 132, 33)


## 使用0来填充nan 

In [54]:
out_data_reshape = np.nan_to_num(out_data_reshape, nan=0)  
out_data_y_reshape = np.nan_to_num(out_data_y_reshape, nan=0)  

# 验证集和测试集的划分

## 对数据进行reshape,以进行数据集的划分

In [55]:
out_data_reshape = np.transpose(out_data_reshape,(1,0,2,3,4))
print('out_data_reshape:',out_data_reshape.shape)
out_data_y_reshape =  np.transpose(out_data_y_reshape,(1,0,2))
print('out_data_y_reshape:',out_data_y_reshape.shape)

out_data_reshape: (132, 101529, 9, 9, 5)
out_data_y_reshape: (132, 101529, 33)


In [56]:
test_num = 120
x_train = out_data_reshape[:test_num]
y_train = out_data_y_reshape[:test_num]

x_test = out_data_reshape[test_num:]
y_test = out_data_y_reshape[test_num:]

In [57]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape 

((120, 101529, 9, 9, 5),
 (120, 101529, 33),
 (12, 101529, 9, 9, 5),
 (12, 101529, 33))

In [58]:
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))

y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))

## 划分训练数据和目标数据

In [59]:
print('x_test.shape:',x_test.shape)
print('y_test.shape',y_test.shape)
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_test.shape: (1218348, 9, 9, 5)
y_test.shape (1218348, 33)
x_train (12183480, 9, 9, 5)
y_train (12183480, 33)


In [60]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=100)

In [61]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8528436, 9, 9, 5)
x_test (1218348, 9, 9, 5)
x_val (3655044, 9, 9, 5)
y_train (8528436, 33)
y_test (1218348, 33)
y_val (3655044, 33)


# 特征归一化

In [62]:
x_train_shape = x_train.shape
y_train_shape = y_train.shape

x_test_shape = x_test.shape
y_test_shape = y_test.shape

x_val_shape = x_val.shape
y_val_shape = y_val.shape

In [63]:
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1,x_train_shape[3])) 
x_test = scaler_f.transform(x_test.reshape(-1,x_test_shape[3]))
x_val = scaler_f.transform(x_val.reshape(-1,x_val_shape[3]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1,y_train_shape[1])) 
y_test = scaler_l.transform(y_test.reshape(-1,y_test_shape[1])) 
y_val = scaler_l.transform(y_val.reshape(-1,y_val_shape[1])) 

In [64]:
x_train = np.reshape(x_train,(-1,9,9,5))
x_test = np.reshape(x_test,(-1,9,9,5))
x_val = np.reshape(x_val,(-1,9,9,5))
y_train = np.reshape(y_train,(-1,33))
y_test = np.reshape(y_test,(-1,33))
y_val = np.reshape(y_val,(-1,33))

In [65]:
print('x_train',x_train.shape)
print('x_test',x_test.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)
print('y_val',y_val.shape)

x_train (8528436, 9, 9, 5)
x_test (1218348, 9, 9, 5)
x_val (3655044, 9, 9, 5)
y_train (8528436, 33)
y_test (1218348, 33)
y_val (3655044, 33)


# 定义网络结构

In [66]:
def ChannelAttention(inputs,in_planes, ratio=2):
    
    avg_out= layers.GlobalAveragePooling2D()
    max_out= layers.GlobalMaxPooling2D()

    fc1 = layers.Dense(in_planes//ratio, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            activation=tf.nn.relu,
                            use_bias=True, bias_initializer='zeros')
    fc2 = layers.Dense(in_planes, kernel_initializer='he_normal',
                            kernel_regularizer=regularizers.l2(5e-4),
                            use_bias=True, bias_initializer='zeros')

    
    avg_out = avg_out(inputs)
    max_out = max_out(inputs)
    out = tf.stack([avg_out, max_out], axis=1)
    out = fc2(fc1(out))
    out = tf.reduce_sum(out, axis=1)
    out = tf.nn.sigmoid(out)
    out = layers.Reshape((1, 1, out.shape[1]))(out)

    return  out*inputs

In [ ]:
def regularized_padded_conv(*args, **kwargs):
    return layers.Conv2D(*args, **kwargs, padding='same', use_bias=False,
                         kernel_initializer='he_normal',
                         kernel_regularizer=regularizers.l2(5e-4))

def SpatialAttention(inputs,kernel_size=5):
    conv1 = regularized_padded_conv(1, kernel_size=kernel_size, strides=1, activation='sigmoid')
    avg_out = tf.reduce_mean(inputs, axis=3)
    max_out = tf.reduce_max(inputs, axis=3)
    out = tf.stack([avg_out, max_out], axis=-1)        
    out = conv1(out)
    return out*inputs

In [ ]:
def cbam_block(inputs, in_planes,ratio=2):
    cbam_feature = ChannelAttention(inputs, in_planes,ratio)
    cbam_feature = SpatialAttention(cbam_feature)
    return cbam_feature

In [67]:
def create_model():
    model_input= Input(shape=x_train.shape[1:])
    output_size = 1
    x = cbam_block(model_input,5)
    x = Conv2D(filters=16, kernel_size=(3, 3),strides=1)(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,16)
    x = Conv2D(filters=32, kernel_size=(3, 3),strides=1)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(rate=0.1)(x)

    x = cbam_block(x,32)
    x = Flatten()(x)

    x = Dense(units=16)(x)
    x = Activation('relu')(x)

    out = Dense(units=output_size)(x)
    
    model = Model(inputs=model_input, outputs=out)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model 

In [ ]:
model = create_model()
model.summary()

## 训练

In [70]:
def train_model_CNN(x_train,y_train,x_val,y_val,name):
    model = create_model()
    model.compile(loss='mse', optimizer='adam')
    checkpoint_path=name
    keras_callbacks   = [
          EarlyStopping(monitor='val_loss', patience=16, mode='min', min_delta=0.001),
          ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
    ]    
    model.fit(x_train, y_train, validation_data=(x_val,y_val),
                    epochs=300, batch_size=10240, verbose=2, callbacks=keras_callbacks)

In [71]:
gc.collect()

2899

In [ ]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]
for i in range(0,33):
    gc.collect()
    train_model_CNN(x_train,y_train[:,i:i+1],x_val,y_val[:,i:i+1],   
                'D:/CNN/sali/CNN_sali_'+str(depth[i])+'m.h5')

## 模型评估

In [86]:
import math
def acc(actual, predicted):
    pred_avg = np.average(predicted)

    act_avg = np.average(actual)
    diff_pred = predicted - pred_avg
    diff_act = actual - act_avg
    numerator = np.mean(np.sum(diff_pred*diff_act, axis=0))
    denominator = math.sqrt(np.mean(np.sum(diff_pred**2, axis=0)) * np.mean(np.sum(diff_act**2, axis=0)))
    ret_val = numerator/denominator
    return (100 * ret_val)

def look_loss(history):
    plt.figure(figsize=(6.3, 2.5), dpi = 100)
    plt.plot(history.history['loss'], label='training data')
    plt.plot(history.history['val_loss'], label='validation data')
    plt.title('Loss')
    plt.ylabel('MSE Loss')
    plt.xlabel('Number of epochs')
    plt.grid()
    plt.legend(loc="upper right")
    plt.show()

def all_estimate(best_model,x_test,y_test):
    testPred_1 = best_model.predict(x_test)
    y_test_p = y_test.reshape(-1,1)
    testPred_p = testPred_1.reshape(-1,1)


    rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
    print('Test RMSE: %.3f' % rmse)

    r2 = r2_score(y_test_p,testPred_p)
    print("R² score: %.4f" % r2)     

def respective_estimate(best_model,x_test,y_test):
    re2_list = []
    rmse_list = []
    for i in range(6):
        testPred = best_model.predict(x_test[i:i+1])
        a = y_test[i:i+1]
        y_test_p = a.reshape(-1,1)
        testPred_p = testPred.reshape(-1,1)
        r2 = r2_score(y_test_p,testPred_p)
        rmse = sqrt(mean_squared_error(y_test_p,testPred_p))
        re2_list.append(r2)
        rmse_list.append(rmse)    
    return re2_list,rmse_list

In [87]:
#look_loss(history)

### 对所有层进行评估

In [90]:
depth = [5,10,15,20,30,40,50,60,70,80,90,100,125,150,175,200,225,250,275,300,350,400,
              500,600,700,800,900,1000,1100,1300,1500,1750,2000]
result = [] 
for i in range(0,33):
    best_model = tf.keras.models.load_model('D:/CNN/sali/CNN_sali_'+str(depth[i])+'m.h5'})
    print('预测第'+str(depth[i])+'m盐度的评价')
    all_estimate(best_model,x_test,y_test[:,i:i+1])
    testPred = best_model.predict(x_test)
    result.append(testPred)

0.079
0.086
0.088
0.095
0.120
0.138
0.168
0.186
0.187
0.195
0.199
0.197
0.234
0.242
0.265
0.291
0.281
0.285
0.267
0.275
0.289
0.265
0.261
0.231
0.236
0.270
0.271
0.262
0.276
0.245
0.251
0.239
0.243


## 反归一化结果评价

In [ ]:
result = np.array(result)
print(result.shape)
result = np.transpose(result,(1,2,0))
print(result.shape)
result = result.reshape(-1,33)
print(result.shape)
y_test = y_test.reshape(-1,33)
print(y_test.shape)

In [101]:
result_unscaled = scaler_l.inverse_transform(result)
y_test_unscaled = scaler_l.inverse_transform(y_test)
result_unscaled.shape,y_test.shape

(33, 1221588, 1)
(1221588, 1, 33)
(1221588, 33)


In [ ]:
for i in range(0,33):
    print('预测第'+str(depth[i])+'m盐度的评价')
    rmse = sqrt(mean_squared_error(y_test_unscaled[:,i],result_unscaled[:,i]))
    print('Test RMSE: %.3f' % rmse)
    r2 = r2_score(y_test_unscaled[:,i],result_unscaled[:,i])
    print('R² score:',r2)

# 预测结果可视化

## 为预测值和真实值填充nan

In [106]:
testPred_with_nan = np.full((161024,12*33), np.nan)  
print(testPred_with_nan.shape)
y_test_with_nan = np.full((161024,12*33), np.nan)  
print(y_test_with_nan.shape)

(161024, 396)
(161024, 396)


In [107]:
result_unscaled_reshape = result_unscaled.reshape(12,-1,33)
print(result_unscaled_reshape.shape)
result_unscaled_reshape = np.transpose(result_unscaled_reshape,(1,0,2))
print(result_unscaled_reshape.shape)
result_unscaled_reshape = result_unscaled_reshape.reshape(-1,12*33)
print(result_unscaled_reshape.shape)

# ==================================================================== #

y_test_unscaled_reshape = y_test_unscaled.reshape(12,-1,33)
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = np.transpose(y_test_unscaled_reshape,(1,0,2))
print(y_test_unscaled_reshape.shape)
y_test_unscaled_reshape = y_test_unscaled_reshape.reshape(-1,12*33)
print(y_test_unscaled_reshape.shape)

(12, 101799, 33)
(101799, 12, 33)
(101799, 396)
(12, 101799, 33)
(101799, 12, 33)
(101799, 396)


In [ ]:
y_test.shape, y_test_reshape.shape

In [108]:
num = 0
for i,nanmask in enumerate(nan_mask):
    if (nanmask ==True):
        testPred_with_nan[i] = result_unscaled_reshape[num]
        y_test_with_nan[i] = y_test_unscaled_reshape[num]
        num = num + 1
        
print(num)

101799


In [109]:
testPred_with_nan = testPred_with_nan.reshape(272, 592,12,33)
testPred_with_nan = np.transpose(testPred_with_nan,(2,3,0,1))

y_test_with_nan = y_test_with_nan.reshape(272, 592,12,33)
y_test_with_nan = np.transpose(y_test_with_nan,(2,3,0,1))

print(testPred_with_nan.shape,y_test_with_nan.shape)

(12, 33, 272, 592) (12, 33, 272, 592)


# 结果可视化

In [110]:
lat = []
lon = []
time = []
lon.append(data_ssh['lon'][4:-4].data)
lat.append(data_ssh['lat'][4:-4].data)

In [112]:
len(lat),len(lat[0]),len(lon),len(lon[0])

(1, 272, 1, 592)

In [113]:
start_year = 2015
end_year =2015
time = []
date_range = pd.date_range(datetime(start_year,1,1),datetime(end_year+1,1,1),freq='1M')
len(date_range)
for i in range(len(date_range)):
    time.append(i)

In [116]:
new_NC = nc.Dataset("D:/CNN/result_sali_CNN_CBAM_5_2000m.nc",
                    'w', format='NETCDF4')
'''
定义维度，后一个参数表示维度的长度，因为是合并的同一个产品的数据，所以是统一
的,注意维度的长度一定要和读入的数据匹配
'''
new_NC.createDimension('lat', len(lat[0]))
new_NC.createDimension('lon', len(lon[0]))
new_NC.createDimension('depth', 33)
new_NC.createDimension('time', len(time))

new_NC.createVariable('lat', 'f', ("lat"))
new_NC.createVariable('lon', 'f', ("lon"))
new_NC.createVariable('depth', 'f', ("depth"))
new_NC.createVariable('Observed_SS', 'f',("time","depth","lat","lon"))
new_NC.createVariable('Estimated_SS', 'f',("time","depth","lat","lon"))

time_var = new_NC.createVariable('time', 'f4',("time"))
time_var.units = 'months since 2015-01-15'
time_var.long_name = 'Months in Monthly Means'
time_var.axis = 'T'

new_NC.variables['lat'][:] = lat[0]
new_NC.variables['lon'][:] = lon[0]
new_NC.variables['time'][:] = np.array(time)
new_NC.variables['depth'][:] = depth

new_NC.variables['Observed_SS'][:]=np.array(y_test_with_nan)
new_NC.variables['Estimated_SS'][:]=np.array(testPred_with_nan)

new_NC.close()